In [43]:
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt 
import pandas as pd
import folium
from folium import plugins

In [2]:
G = nx.Graph()

# NODES of GRAPH Network

In [3]:
with open(r'USA-road-d.CAL.co','r') as f1:
    for line in f1:
        if line[0] == 'v':
            n,la,lo = list(map(int, line[2:].split()))
            G.add_node(n,latitude = lo/1000000,longitude = la/1000000)

In [4]:
G.nodes[5]

{'latitude': 34.042614, 'longitude': -114.3473}

In [55]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 1890815\nNumber of edges: 2315222\nAverage degree:   2.4489'

# Edges (Distance) of Graph Network

In [5]:
adj = defaultdict(set)
with open(r'USA-road-d.CAL.gr','r') as f:
    for line in f:
        if line[0] == 'a':
            n1,n2, d =  list(map(int, line[2:].split()))
            G.add_edge(n1,n2,distance = d,weight = 1)
            adj[n1].add(n2)
            adj[n2].add(n1)

In [ ]:
nx.info(G)

In [54]:
G.edges[5,6]

{'distance': 18133, 'weight': 1, 'time': 45332}

# Edges (Time) of Graph Network

In [12]:
with open(r'USA-road-t.CAL.gr','r') as f2:
    for line in f2:
        if line[0] == 'a':
            n1,n2, t =  list(map(int, line[2:].split()))
            G.add_edge(n1,n2,time = t)

In [14]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 1890815\nNumber of edges: 2315222\nAverage degree:   2.4489'

In [56]:
G.edges[5,6]

{'distance': 18133, 'weight': 1, 'time': 45332}

In [15]:
nx.has_path(G, source=5, target=6)

True

In [17]:
nx.dijkstra_path(G, source=2244, target=1050368)

[2244, 5, 6, 1050368]

In [57]:
# Making Nodes in Network into Dataframe
#NodesDataframe = pd.DataFrame([G.nodes[i] for i in range(1,len(G.nodes))])
#NodesDataframe.head()

# Map Function

In [77]:
# Map

# V is the input node or start node

def map(nodelst):
      
    pos = G.nodes[nodelst[0]]# for the start point
    
    # creating the Map focusing on the start point

    vismap = folium.Map(location=[pos['latitude'], pos['longitude']], zoom_start=10)
    
    # creating a marker of HOme on the start point
    
    folium.Marker(location=[(pos['latitude']),(pos['longitude'])],
                  icon=folium.Icon(color='red', icon='home'), popup = (nodelst[0])).add_to(vismap)

    # creating a marker on the rest of the point
    
    for i in range (len(nodelst)-1):
        pos = (G.nodes[nodelst[i+1]])
        folium.Marker(location=[(pos['latitude']),(pos['longitude'])],popup = (nodelst[i+1])).add_to(vismap)
    
    return vismap

# Map Route Function

In [76]:
# a is the list of nodes
# map_name is the map with nodes already generated


def map_routes(nodelst,map_name):
    cordlst = 0
    cordlst = []
    # making list of all coordinates of the NODES avaialble in a
    for i in nodelst:
        cordlst.append(list(G.nodes[i].values())) 
    plugins.AntPath(cordlst).add_to(map_name)
       
    return map_name

# Function 1

In [78]:
# v is the input Node
# P is the type of weight distance or time
# d is the thrushhold
# neigh [] is a list of neighbour

def fun1(v,p,d,neigh = []):
    neigh += [v]
    adjs = adj[v]
    for node in adjs:
        if G[v][node][p] <= d and node not in neigh:
            new_d = d-G[v][node][p]
            neigh = fun1(node,p,new_d,neigh)
    return neigh

# Visualization F1 - Front End

In [ ]:

v = (int(input('Enter the node number: ')))
p = (input('Enter the type of weight, Either distance or time: '))
d = int(input('Enter the thrushhold for the type of weight: '))
nodelst = fun1(v,p,d)
map(nodelst).save('F1map.html')
#m.save('F1map.html')

# Visualization F3 - Front End

In [ ]:
s = int(input("Enter the Home Node: "))
t = int(input("Enter the departure Node: "))

nodelst = nx.dijkstra_path(G, source=s, target=t)
c = map(nodelst)
map_routes(nodelst,c).save('F3map.html')